In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup
import pandas as pd
from datetime import date

In [11]:
driver = webdriver.Chrome('chromedriver')
driver.get('htmls/cristiano.html')

InvalidArgumentException: Message: invalid argument
  (Session info: chrome=95.0.4638.69)
Stacktrace:
#0 0x5568a47d5f03 <unknown>
#1 0x5568a454adef <unknown>
#2 0x5568a4537131 <unknown>
#3 0x5568a4535cd6 <unknown>
#4 0x5568a4535e13 <unknown>
#5 0x5568a454c8fa <unknown>
#6 0x5568a45b0cff <unknown>
#7 0x5568a459d4e2 <unknown>
#8 0x5568a45b0459 <unknown>
#9 0x5568a459d3d3 <unknown>
#10 0x5568a45738b3 <unknown>
#11 0x5568a4574885 <unknown>
#12 0x5568a4802f51 <unknown>
#13 0x5568a481643b <unknown>
#14 0x5568a4817713 <unknown>
#15 0x5568a4803e3e <unknown>
#16 0x5568a4817bf4 <unknown>
#17 0x5568a47f8e97 <unknown>
#18 0x5568a4830758 <unknown>
#19 0x5568a48308d8 <unknown>
#20 0x5568a484af56 <unknown>
#21 0x7f760c9526db <unknown>


In [7]:
# Fazendo Login

username = driver.find_element(By.CSS_SELECTOR, "input[name='dashboard_email']")
password = driver.find_element(By.CSS_SELECTOR, "input[name='dashboard_password']")

username.clear()
password.clear()

username.send_keys('wendeline2@gmail.com')
password.send_keys('Wen@011235813')

log_in = driver.find_element(By.CSS_SELECTOR, "input[type='submit']").click()

In [10]:
driver.get('https://socialblade.com/instagram/user/cristiano')

In [22]:
soup = open('htmls/cristiano.html').read()

In [24]:
soup = BeautifulSoup(soup, 'html.parser')

In [27]:
soup

<!DOCTYPE html>

<html class="fa-events-icons-ready"><head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<title>Cristiano Ronaldo's Instagram Stats Summary Profile (Social Blade Instagram Statistics)</title>
<meta content="Utilize SocialBlade.com to check your Instagram Stats and Instagram Followers while tracking your progress. SocialBlade is a premiere Instagram community where you can chat with other Instagram users." name="description"/>
<meta content="cristiano, Instagram Summary Profile, Instagram Followers, Instagram Statistics, Social Blade, SocialBlade, SocialBlade Stats, Instagram Stats, Instagram" name="keywords"/>
<meta charset="utf-8"/>
<meta content="website" property="og:type"/>
<meta content="Utilize SocialBlade.com to check your Instagram Stats and Instagram Followers while tracking your progress. SocialBlade is a premiere Instagram community where you can chat with other Instagram users." property="og:description"/>
<meta content="summary_large

In [28]:
def instaData(user, today):
    # Puxando a página:
    
    url = 'htmls/' + user + '.html'

    # Tratando a pagina
    html = open(url).read()

    # Invocando o BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Puxando o a tabela que será utilizada

    social_blade_container = soup.find('div', {'id' : 'socialblade-user-content'})
    style1 = {'style' : 'width: 860px; height: 30px; line-height: 30px; background: #f8f8f8;; border-bottom: 1px solid #ddd; padding: 0px 20px; color:#444; font-size: 9pt;'}
    style2 = {'style' : 'width: 860px; height: 30px; line-height: 30px; background: #fafafa; border-bottom: 1px solid #ddd; padding: 0px 20px; color:#444; font-size: 9pt;'}
    data = social_blade_container.find('div', style = "width: 880px; background: #fff; padding: 0px; color:#444; font-size: 10pt;")
    rows1 = data.findAll('div', style1)
    rows2 = data.findAll('div', style2)
    rows = []
    for row in range(len(rows1)):
        rows.append(rows1[row])
        rows.append(rows2[row])

    n_rows = len(rows)

    # Declarando variáveis para alimentar o dataframe:

    dates = []
    days = []
    followers = []
    following = []
    posts = []


    # Puxando lista de datas
    for item in range(n_rows):
        date = rows[item].findAll('div')[1].get_text()
        dates.append(date)

    # Puxando lista de dias da semana
    for item in range(n_rows):
        day = rows[item].findAll('div')[2].get_text()
        days.append(day)

    # Puxando lista do numero de seguidores
    for item in range(n_rows):
        follower = rows[item].findAll('div')[5].get_text().split()[0]
        follower = follower.replace(',','')
        followers.append(follower)

    # Puxando lista do número de pessoas que está seguindo
    for item in range(n_rows):
        follow = rows[item].findAll('div')[8].get_text()
        follow = follow.replace(',','')
        following.append(follow)

    # Puxando lista do número de posts realizados
    for item in range(n_rows):
        post = rows[item].findAll('div')[11].get_text()
        post = post.replace(',','')
        posts.append(post)

    # Criando o dataframe:

    # Criando o dataframe de cada coluna
    df_date = pd.DataFrame(dates, columns = ['data'])
    df_days = pd.DataFrame(days, columns = ['days'])
    df_followers = pd.DataFrame(followers, columns = ['followers'])
    df_following = pd.DataFrame(following, columns = ['following'])
    df_posts = pd.DataFrame(posts, columns = ['posts'])

    # Concatenando tudo em um só dataframe
    insta_data = pd.concat([df_date, df_days, df_followers, df_following, df_posts], axis = 1)


    # Tratando os dados do dataframe:

    # Convertendo a coluna date para o tipo 'datetime'
    #insta_data['data'] = pd.to_datetime(insta_data['data'])

    # Traduzindo os dados da coluna 'days'
    weekdays = {'Thu' : 'Qui'
                , 'Fri' : 'Sex'
                , 'Sat' : 'Sab'
                , 'Sun' : 'Dom'
                , 'Mon' : 'Seg'
                , 'Wed' : 'Qua'
                , 'Tue' : 'Ter'}

    insta_data['days'] = insta_data.days.map(weekdays)

    # Traduzindo os nomes das colunas
    columns = {'data' : 'data'
               , 'days' : 'dia'
               , 'followers' :'seguidores'
               , 'following' :'seguindo'
               , 'posts' : 'posts'}

    insta_data = insta_data.rename(columns = columns)


    # Gerando arquivo .csv
    #insta_data.to_csv('participantes/dados/' + today + '/' + user + '.csv', sep = ',', index = False)
    
    return insta_data

In [30]:
today = date.today().strftime("%d-%m-%Y")

In [31]:
instaData('cristiano', today)

AttributeError: 'NoneType' object has no attribute 'findAll'

In [32]:
soup

<!DOCTYPE html>

<html class="fa-events-icons-ready"><head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<title>Cristiano Ronaldo's Instagram Stats Summary Profile (Social Blade Instagram Statistics)</title>
<meta content="Utilize SocialBlade.com to check your Instagram Stats and Instagram Followers while tracking your progress. SocialBlade is a premiere Instagram community where you can chat with other Instagram users." name="description"/>
<meta content="cristiano, Instagram Summary Profile, Instagram Followers, Instagram Statistics, Social Blade, SocialBlade, SocialBlade Stats, Instagram Stats, Instagram" name="keywords"/>
<meta charset="utf-8"/>
<meta content="website" property="og:type"/>
<meta content="Utilize SocialBlade.com to check your Instagram Stats and Instagram Followers while tracking your progress. SocialBlade is a premiere Instagram community where you can chat with other Instagram users." property="og:description"/>
<meta content="summary_large

In [33]:

social_blade_container = soup.find('div', {'id' : 'socialblade-user-content'})
style1 = {'style' : 'width: 860px; height: 30px; line-height: 30px; background: #f8f8f8;; border-bottom: 1px solid #ddd; padding: 0px 20px; color:#444; font-size: 9pt;'}
style2 = {'style' : 'width: 860px; height: 30px; line-height: 30px; background: #fafafa; border-bottom: 1px solid #ddd; padding: 0px 20px; color:#444; font-size: 9pt;'}
data = social_blade_container.find('div', style = "width: 880px; background: #fff; padding: 0px; color:#444; font-size: 10pt;")

In [38]:
social_blade_container.find('div', style = "width: 880px; background: #fff; padding: 0px; color:#444; font-size: 10pt;")

In [36]:
rows1 = data.findAll('div', style1)

AttributeError: 'NoneType' object has no attribute 'findAll'